In [ ]:
ls /workspace/tracking_dtat

In [ ]:
import torch
ckpt = torch.load('/workspace/tracking_datasets/stark/checkpoints/train/stark_st2/baseline_got10k_only/STARKST_ep0050.pth.tar')

In [ ]:
ckpt.keys()

In [ ]:
ckpt['net'].keys()

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import torch
ckpt = torch.load('/workspace/tracking_datasets/stark/checkpoints/train/stark_sparse/baseline_got10k_only/STARKST_ep0030.pth.tar')

In [ ]:
ckpt.keys()

In [ ]:
ckpt['net'].keys()

In [ ]:
ckpt['net']['backbone.0.body.layer1.0.conv1.weight']

In [ ]:
ckpt['net']['backbone.0.body.layer2.0.bn2.weight']

In [ ]:
ckpt['net']['backbone.0.body.layer1.0.bn3.weight']

In [ ]:
ckpt1 = torch.load('/workspace/tracking_datasets/STARKST_ep0050.pth.tar')

In [ ]:
ckpt1['net']['backbone.0.body.layer2.0.bn2.weight']

In [ ]:
# ckpt['net'] ckpt1['net']
for i in ckpt['net'].keys():
#     print(i)
    try:
        if 'bn' in i:
           print((ckpt['net'][i] - ckpt1['net'][i]).sum(),'      ',i)
    except:
        print(i)
        continue

In [ ]:
# ckpt['net'] ckpt1['net']
for i in ckpt['net'].keys():
#     print(i)
    try:
        if 'bn' in i:
           print((ckpt['net'][i] - ckpt1['net'][i]).sum(),'      ',i)
    except:
        print(i)
        continue

In [ ]:
ckpt1 = torch.load('/workspace/tracking_datasets/STARKST_ep0050.pth.tar')
ckpt1.keys()

In [ ]:
ckpt1['net'].keys()

In [ ]:
ckpt1['net']['transformer.encoder.layers.0.self_attn.in_proj_bias'].shape

In [ ]:
import torch.nn as nn

In [ ]:
# multihead_attn = nn.MultiheadAttention(512,8)

In [ ]:
# multihead_attn.state_dict().keys()

In [ ]:
multihead_attn.state_dict()['in_proj_bias'].shape

In [ ]:
# import torch.tensor as Tensor
# def _in_projection_packed(q,k,v,w,b):
#     r"""
#     Performs the in-projection step of the attention operation, using packed weights.
#     Output is a triple containing projection tensors for query, key and value.
#     Args:
#         q, k, v: query, key and value tensors to be projected. For self-attention,
#             these are typically the same tensor; for encoder-decoder attention,
#             k and v are typically the same tensor. (We take advantage of these
#             identities for performance if they are present.) Regardless, q, k and v
#             must share a common embedding dimension; otherwise their shapes may vary.
#         w: projection weights for q, k and v, packed into a single tensor. Weights
#             are packed along dimension 0, in q, k, v order.
#         b: optional projection biases for q, k and v, packed into a single tensor
#             in q, k, v order.
#     Shape:
#         Inputs:
#         - q: :math:`(..., E)` where E is the embedding dimension
#         - k: :math:`(..., E)` where E is the embedding dimension
#         - v: :math:`(..., E)` where E is the embedding dimension
#         - w: :math:`(E * 3, E)` where E is the embedding dimension
#         - b: :math:`E * 3` where E is the embedding dimension
#         Output:
#         - in output list :math:`[q', k', v']`, each output tensor will have the
#             same shape as the corresponding input tensor.
#     """
#     E = q.size(-1)
#     if k is v:
#         if q is k:
#             # self-attention
#             return linear(q, w, b).chunk(3, dim=-1)
#         else:
#             # encoder-decoder attention
#             w_q, w_kv = w.split([E, E * 2])
#             if b is None:
#                 b_q = b_kv = None
#             else:
#                 b_q, b_kv = b.split([E, E * 2])
#             return (linear(q, w_q, b_q),) + linear(k, w_kv, b_kv).chunk(2, dim=-1)
#     else:
#         w_q, w_k, w_v = w.chunk(3)
#         if b is None:
#             b_q = b_k = b_v = None
#         else:
#             b_q, b_k, b_v = b.chunk(3)
#         return linear(q, w_q, b_q), linear(k, w_k, b_k), linear(v, w_v, b_v)

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., num_patches=197):
        super().__init__()
        self.num_heads = num_heads
        self.num_patches = num_patches
        head_dim = dim // num_heads
        self.dim = dim
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, q,k,v):
        B, N, C = q.shape
        
        q = F.linear(q,self.qkv.weight.data[:self.dim,:],self.qkv.bias.data[:self.dim]).reshape(B, N, 1, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        k = F.linear(k,self.qkv.weight.data[self.dim:self.dim*2,:],self.qkv.bias.data[self.dim:self.dim*2]).reshape(B, N, 1, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        v = F.linear(v,self.qkv.weight.data[self.dim*2:self.dim*3,:],self.qkv.bias.data[self.dim*2:self.dim*3]).reshape(B, N, 1, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        
        q = q[0]
        k = k[0]
        v = v[0]
#         qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
#         q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
#         print(attn)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [ ]:
# a_t = Attention(256,8,qkv_bias=True)
# a_t.state_dict().keys()

In [ ]:
# a_t.state_dict()['qkv.weight'].shape

In [ ]:
x = torch.ones((2,1,8))
y = torch.ones((2,1,8))*2
z = torch.ones((2,1,8))*3

# x = torch.rand((2,1,8))
# y = x*2
# z = x*3
em = torch.cat((x,y,z),dim=1)
print(em.shape)

In [ ]:
a_t = Attention(8,8,qkv_bias=True)
a_t(em,em,em).shape

In [ ]:
a_t(em,em,em)

In [ ]:
multihead_attn = nn.MultiheadAttention(8,8)
multihead_attn(em,em,em)[0].shape

In [ ]:
multihead_attn(em,em,em)[0]

In [ ]:
ckpt_ma = multihead_attn.state_dict()

In [ ]:
ckpt_aa = a_t.state_dict()

In [ ]:
for i in range(len(ckpt_ma.keys())):
    ckpt_aa[list(ckpt_aa.keys())[i]] = ckpt_ma[list(ckpt_ma.keys())[i]]

In [ ]:
ckpt_aa

In [ ]:
ckpt_ma

In [ ]:
multihead_attn.load_state_dict(ckpt_ma)
a_t.load_state_dict(ckpt_aa)

In [ ]:
multihead_attn.eval()
a_t.eval()

In [ ]:
em.shape

In [ ]:
em[:,0,:].shape

In [ ]:
x.shape

In [ ]:
multihead_attn(x.permute(1,0,2),y.permute(1,0,2),z.permute(1,0,2))[0].permute(1,0,2)

In [ ]:
# multihead_attn(em,em,em)[0]

In [ ]:
a_t(x,y,z)

In [ ]:
ckpt1.keys()

In [ ]:
stm = torch.load('/workspace/STMTrack/models/googlenet/inception_v3_google-1a9a5a14-961cad7697695cca7d9ca4814b17a88d.pth')

In [ ]:
stm.keys()

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
class MultiHeadAttention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., num_patches=197):
        super().__init__()
        self.num_heads = num_heads
        self.num_patches = num_patches
        head_dim = dim // num_heads
        self.dim = dim
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, q,k,v,attn_mask=None):
        
        q = q.permute(1,0,2)
        k = k.permute(1,0,2)
        v = v.permute(1,0,2)
        
        
        B,N_q,N_k,N_v,C = q.shape[0],q.shape[1],k.shape[1],v.shape[1],q.shape[2]
        
        
        q = F.linear(q,self.qkv.weight.data[:self.dim,:],self.qkv.bias.data[:self.dim]).reshape(B, N_q, 1, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)[0]
        k = F.linear(k,self.qkv.weight.data[self.dim:self.dim*2,:],self.qkv.bias.data[self.dim:self.dim*2]).reshape(B, N_k, 1, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)[0]
        v = F.linear(v,self.qkv.weight.data[self.dim*2:self.dim*3,:],self.qkv.bias.data[self.dim*2:self.dim*3]).reshape(B, N_v, 1, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)[0]

        
        print(q.shape,k.shape,v.shape)
        if attn_mask is not None:
            attn_mask = attn_mask.to(torch.bool)

            attn_mask = attn_mask.view(B, 1, 1, N_k).expand(-1, self.num_heads, -1, -1).reshape(B*self.num_heads, 1, N_k)

            new_attn_mask = torch.zeros_like(attn_mask, dtype=q.dtype)
            new_attn_mask.masked_fill_(attn_mask, float("-inf"))
            attn_mask = new_attn_mask
        
        
        if attn_mask is not None:
            attn = torch.baddbmm(attn_mask, q.reshape(B*self.num_heads,N_q,C // self.num_heads), k.reshape(B*self.num_heads,N_k,C // self.num_heads).transpose(-2, -1)) * self.scale
            attn = attn.reshape(B,self.num_heads,N_q,N_k)
            
        else:
            attn = (q @ k.transpose(-2, -1)) * self.scale
            
        attn = attn.softmax(dim=-1)
        
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N_q, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        x = x.permute(1,0,2)
        return x

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch
class MultiHeadAttention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., num_patches=197):
        super().__init__()
        self.num_heads = num_heads
        self.num_patches = num_patches
        head_dim = dim // num_heads
        self.dim = dim
        self.scale = qk_scale or head_dim ** -0.5

        self.q = nn.Linear(dim, dim, bias=qkv_bias)
        self.v = nn.Linear(dim, dim, bias=qkv_bias)
        self.k = nn.Linear(dim, dim, bias=qkv_bias)
        
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, q,k,v,attn_mask=None):
        
        q = q.permute(1,0,2)
        k = k.permute(1,0,2)
        v = v.permute(1,0,2)
        
        
        B,N_q,N_k,N_v,C = q.shape[0],q.shape[1],k.shape[1],v.shape[1],q.shape[2]
        
        
        q = self.q(q).reshape(B, N_q, 1, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)[0]
        k = self.k(k).reshape(B, N_k, 1, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)[0]
        v = self.v(v).reshape(B, N_v, 1, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)[0]

        
        print(q.shape,k.shape,v.shape)
        if attn_mask is not None:
            attn_mask = attn_mask.to(torch.bool)

            attn_mask = attn_mask.view(B, 1, 1, N_k).expand(-1, self.num_heads, -1, -1).reshape(B*self.num_heads, 1, N_k)

            new_attn_mask = torch.zeros_like(attn_mask, dtype=q.dtype)
            new_attn_mask.masked_fill_(attn_mask, float("-inf"))
            attn_mask = new_attn_mask
        
        
        if attn_mask is not None:
            attn = torch.baddbmm(attn_mask, q.reshape(B*self.num_heads,N_q,C // self.num_heads), k.reshape(B*self.num_heads,N_k,C // self.num_heads).transpose(-2, -1)) * self.scale
            attn = attn.reshape(B,self.num_heads,N_q,N_k)
            
        else:
            attn = (q @ k.transpose(-2, -1)) * self.scale
            
        attn = attn.softmax(dim=-1)
        
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N_q, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        x = x.permute(1,0,2)
        return x

In [6]:
cus = MultiHeadAttention(16,8,qkv_bias=True)
att = nn.MultiheadAttention(16,8)

In [7]:
cus.state_dict().keys()

odict_keys(['q.weight', 'q.bias', 'v.weight', 'v.bias', 'k.weight', 'k.bias', 'proj.weight', 'proj.bias'])

In [8]:
att.state_dict().keys()

odict_keys(['in_proj_weight', 'in_proj_bias', 'out_proj.weight', 'out_proj.bias'])

In [9]:
cus_ckpt = cus.state_dict()
att_ckpt = att.state_dict()

torch.Size([48, 16])

In [15]:
cus_ckpt['q.weight'] = att_ckpt['in_proj_weight'][:16,:]
cus_ckpt['k.weight'] = att_ckpt['in_proj_weight'][16:32,:]
cus_ckpt['v.weight'] = att_ckpt['in_proj_weight'][32:48,:]

cus_ckpt['q.bias'] = att_ckpt['in_proj_bias'][:16]
cus_ckpt['k.bias'] = att_ckpt['in_proj_bias'][16:32]
cus_ckpt['v.bias'] = att_ckpt['in_proj_bias'][32:48]

cus_ckpt['proj.weight'] = att_ckpt['out_proj.weight']
cus_ckpt['proj.bias'] = att_ckpt['out_proj.bias']

In [ ]:
for i in range(len(cus_ckpt.keys())):
    cus_ckpt[list(cus_ckpt.keys())[i]] = att_ckpt[list(att_ckpt.keys())[i]]

In [16]:
cus.load_state_dict(cus_ckpt)
att.load_state_dict(att_ckpt)

<All keys matched successfully>

In [17]:
q = torch.rand((4,2,16))
k = torch.rand((8,2,16))
v = torch.rand((8,2,16))

att_mask = torch.randint(0, 2, (2,8))

In [ ]:
att_mask

In [20]:
import numpy as np
cus(q,k,v,None)  #bs,nhead,seq_len,head_size

torch.Size([2, 8, 4, 2]) torch.Size([2, 8, 8, 2]) torch.Size([2, 8, 8, 2])


tensor([[[-2.3039e-01,  1.3611e-01, -3.8231e-02, -1.8606e-01,  7.8508e-02,
          -1.5547e-01, -1.0202e-01,  1.4707e-01, -2.2271e-01, -6.0652e-02,
           5.9124e-03, -1.1733e-01,  2.2855e-01, -7.1308e-02, -9.2387e-03,
          -6.3134e-02],
         [-1.3239e-01,  1.8836e-01, -5.6286e-02, -1.6565e-01,  1.3476e-01,
          -7.2032e-02, -1.4834e-01,  9.6906e-02, -2.3595e-01,  1.2915e-02,
           6.3564e-04, -5.1516e-02,  1.9877e-01, -9.3561e-02, -3.8331e-03,
           6.8967e-04]],

        [[-2.3067e-01,  1.3946e-01, -3.8197e-02, -1.8524e-01,  8.0181e-02,
          -1.5689e-01, -1.0177e-01,  1.5134e-01, -2.2213e-01, -5.9793e-02,
           2.6337e-03, -1.2083e-01,  2.2955e-01, -7.1954e-02, -1.2970e-02,
          -6.4759e-02],
         [-1.3532e-01,  1.8466e-01, -5.6312e-02, -1.6681e-01,  1.3347e-01,
          -7.1122e-02, -1.4797e-01,  9.5620e-02, -2.3218e-01,  1.0999e-02,
           1.3105e-03, -4.8062e-02,  2.0119e-01, -9.2952e-02,  5.3359e-04,
           3.1482e-03]],



In [21]:
import numpy as np
cus(q,k,v,att_mask)  #bs,nhead,seq_len,head_size

torch.Size([2, 8, 4, 2]) torch.Size([2, 8, 8, 2]) torch.Size([2, 8, 8, 2])


tensor([[[-0.2296,  0.1092, -0.0378, -0.1628,  0.0602, -0.1009, -0.0860,
           0.1388, -0.1583, -0.0290,  0.0086, -0.1117,  0.2411, -0.1210,
           0.0429, -0.0440],
         [-0.1784,  0.1857, -0.0227, -0.1172,  0.1376, -0.1487, -0.1750,
           0.0757, -0.2508,  0.0054,  0.0099, -0.0773,  0.1493, -0.0864,
          -0.0455,  0.0017]],

        [[-0.2307,  0.1094, -0.0371, -0.1627,  0.0602, -0.1042, -0.0857,
           0.1411, -0.1589, -0.0278,  0.0055, -0.1146,  0.2418, -0.1201,
           0.0384, -0.0451],
         [-0.1803,  0.1793, -0.0297, -0.1227,  0.1416, -0.1496, -0.1742,
           0.0762, -0.2485, -0.0013,  0.0114, -0.0698,  0.1559, -0.0885,
          -0.0457,  0.0036]],

        [[-0.2302,  0.1119, -0.0372, -0.1625,  0.0608, -0.1014, -0.0871,
           0.1430, -0.1583, -0.0277,  0.0057, -0.1148,  0.2433, -0.1220,
           0.0407, -0.0465],
         [-0.1820,  0.1818, -0.0264, -0.1165,  0.1388, -0.1527, -0.1756,
           0.0687, -0.2475,  0.0007,  0.0159, -0

In [18]:
att(q,k,v,key_padding_mask = None)[0]

tensor([[[-2.3039e-01,  1.3611e-01, -3.8231e-02, -1.8606e-01,  7.8508e-02,
          -1.5547e-01, -1.0202e-01,  1.4707e-01, -2.2271e-01, -6.0652e-02,
           5.9123e-03, -1.1733e-01,  2.2855e-01, -7.1308e-02, -9.2388e-03,
          -6.3134e-02],
         [-1.3239e-01,  1.8836e-01, -5.6286e-02, -1.6565e-01,  1.3476e-01,
          -7.2032e-02, -1.4834e-01,  9.6906e-02, -2.3595e-01,  1.2915e-02,
           6.3564e-04, -5.1516e-02,  1.9877e-01, -9.3561e-02, -3.8331e-03,
           6.8967e-04]],

        [[-2.3067e-01,  1.3946e-01, -3.8197e-02, -1.8524e-01,  8.0181e-02,
          -1.5689e-01, -1.0177e-01,  1.5134e-01, -2.2213e-01, -5.9793e-02,
           2.6337e-03, -1.2083e-01,  2.2955e-01, -7.1954e-02, -1.2970e-02,
          -6.4759e-02],
         [-1.3532e-01,  1.8466e-01, -5.6312e-02, -1.6681e-01,  1.3347e-01,
          -7.1122e-02, -1.4797e-01,  9.5620e-02, -2.3218e-01,  1.0999e-02,
           1.3105e-03, -4.8062e-02,  2.0119e-01, -9.2952e-02,  5.3361e-04,
           3.1482e-03]],



In [19]:
att(q,k,v,key_padding_mask = att_mask)[0]

tensor([[[-0.2296,  0.1092, -0.0378, -0.1628,  0.0602, -0.1009, -0.0860,
           0.1388, -0.1583, -0.0290,  0.0086, -0.1117,  0.2411, -0.1210,
           0.0429, -0.0440],
         [-0.1784,  0.1857, -0.0227, -0.1172,  0.1376, -0.1487, -0.1750,
           0.0757, -0.2508,  0.0054,  0.0099, -0.0773,  0.1493, -0.0864,
          -0.0455,  0.0017]],

        [[-0.2307,  0.1094, -0.0371, -0.1627,  0.0602, -0.1042, -0.0857,
           0.1411, -0.1589, -0.0278,  0.0055, -0.1146,  0.2418, -0.1201,
           0.0384, -0.0451],
         [-0.1803,  0.1793, -0.0297, -0.1227,  0.1416, -0.1496, -0.1742,
           0.0762, -0.2485, -0.0013,  0.0114, -0.0698,  0.1559, -0.0885,
          -0.0457,  0.0036]],

        [[-0.2302,  0.1119, -0.0372, -0.1625,  0.0608, -0.1014, -0.0871,
           0.1430, -0.1583, -0.0277,  0.0057, -0.1148,  0.2433, -0.1220,
           0.0407, -0.0465],
         [-0.1820,  0.1818, -0.0264, -0.1165,  0.1388, -0.1527, -0.1756,
           0.0687, -0.2475,  0.0007,  0.0159, -0

In [ ]:
import torc
ckpt = torch.load('/workspace/tracking_datasets/stark/try/checkpoints/train/stark_sparse/baseline_got10k_only_sparse/STARKST_ep0001.pth.tar')

In [ ]:
ckpt.keys()

In [ ]:
ckpt['net']['transformer.encoder.layers.0.mlp.zeta']

In [ ]:
ckpt['net']['transformer.encoder.layers.0.self_attn.zeta']

In [ ]:
import torch
ckpt1 = torch.load('/workspace/tracking_datasets/stark/try/checkpoints/train/stark_sparse/baseline_got10k_only_sparse/STARKST_ep0001.pth.tar')

In [ ]:
ckpt1['net']['transformer.encoder.layers.0.self_attn.qkv.weight']

In [ ]:
import torch
ckpt2 = torch.load('/workspace/tracking_datasets/STARKST_ep0050.pth.tar')

In [ ]:
ckpt2['net']['transformer.encoder.layers.0.self_attn.in_proj_weight']

In [ ]:
ckpt1['net'].keys()